# Tratamiento de datos faltantes
Generalmente se consideran 3 opciones básicas ante datos faltantes:
- Mantener los puntos de datos faltantes
- Eliminar los puntos de datos faltantes (puede implicar eliminar la fila completa)
- Rellenarlos con algún valor

### Mantener los puntos de datos faltantes
Depende del modelo de analítica que lo pueda manejar

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("datosfaltantes").getOrCreate()

In [0]:
ruta = "dbfs:/FileStore/shared_uploads/jgamarramoreno@gmail.com/ContainsNull.csv"

In [0]:
df = spark.read.csv(ruta,header=True,inferSchema=True)

In [0]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Eliminar los puntos de datos faltantes

Se usa el comando drop con los siguientes parámetros:

df.na.drop(how='any',thresh=None,subset=None)

- Parámetro 'how': 'any' o 'all'
  - 'any', elimina la fila si este contiene algún nulo
  - 'all', elimina la fila si todos sus valores son nulos
- Parámetro 'thresh': un entero por defecto ninguno
  - Si se establece un valor, elimina las filas que tienen una cantidad de valores nulos mayor o igual que el valor establecido
  - Sobrescribe el parámetro 'how'
- Parámetro subset
  - lista opcional de nombres de columna a considerar para eliminar la fila

In [0]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [0]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [0]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [0]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [0]:
df.na.drop(subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Rellenarlos con algún valor

In [0]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [0]:
df.na.fill("Nuevo Valor").show()

+----+-----------+-----+
|  Id|       Name|Sales|
+----+-----------+-----+
|emp1|       John| null|
|emp2|Nuevo Valor| null|
|emp3|Nuevo Valor|345.0|
|emp4|      Cindy|456.0|
+----+-----------+-----+



In [0]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



Usualmente se especifica con subset que columnas se quieren llenar con los valores de reemplazo

In [0]:
df.na.fill('Sin nombre',subset=['Name']).show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|Sin nombre| null|
|emp3|Sin nombre|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



Llenar las filas con el valor promedio

In [0]:
from pyspark.sql.functions import mean

In [0]:
val_prom = df.select(mean(df['Sales'])).collect()

In [0]:
val_prom

Out[18]: [Row(avg(Sales)=400.5)]

In [0]:
prom_ventas = val_prom[0][0]

In [0]:
df.na.fill(prom_ventas,["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [0]:
# en una sola línea de código
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

